# Stock NeurIPS2018 Part 1. Data
This series is a reproduction of paper *the process in the paper Practical Deep Reinforcement Learning Approach for Stock Trading*. 

This is the first part of the NeurIPS2018 series, introducing how to use FinRL to fetch and process data that we need for ML/RL trading.

Other demos can be found at the repo of [FinRL-Tutorials]((https://github.com/AI4Finance-Foundation/FinRL-Tutorials)).

# tock NeurIPS2018 Part 1. 데이터
이 시리즈는 Practical Deep Reinforcement Learning Approach for Stock Trading 논문에서 제시한 과정을 재현한 것입니다.

NeurIPS2018 시리즈의 첫 번째 파트로, ML/RL 기반의 주식 트레이딩을 위해 필요한 데이터를 FinRL을 사용해 수집하고 전처리하는 방법을 소개합니다.

기타 데모들은 FinRL-Tutorials 저장소에서 확인할 수 있습니다.

# Part 1. Install Packages

In [ ]:
## install finrl library
# !pip install git+https://github.com/AI4Finance-Foundation/FinRL.git

In [1]:
import pandas as pd
import yfinance as yf

from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl import config_tickers
from finrl.config import INDICATORS
from finrl.config import *
import itertools

# Part 2. Fetch data

[yfinance](https://github.com/ranaroussi/yfinance) is an open-source library that provides APIs fetching historical data form Yahoo Finance. In FinRL, we have a class called [YahooDownloader](https://github.com/AI4Finance-Foundation/FinRL/blob/master/finrl/meta/preprocessor/yahoodownloader.py) that use yfinance to fetch data from Yahoo Finance.

**OHLCV**: Data downloaded are in the form of OHLCV, corresponding to **open, high, low, close, volume,** respectively. OHLCV is important because they contain most of numerical information of a stock in time series. From OHLCV, traders can get further judgement and prediction like the momentum, people's interest, market trends, etc.

yfinance는 야후 파이낸스(Yahoo Finance)에서 과거 데이터를 가져올 수 있는 API를 제공하는 오픈소스 라이브러리입니다.
FinRL에서는 YahooDownloader라는 클래스를 사용하여 yfinance를 통해 데이터를 가져옵니다.

OHLCV: 다운로드된 데이터는 OHLCV 형식으로 제공되며, 각각 **시가(Open), 고가(High), 저가(Low), 종가(Close), 거래량(Volume)**을 의미합니다.
OHLCV 데이터는 시계열에서 주식의 대부분의 수치 정보를 담고 있어 매우 중요합니다. 트레이더들은 OHLCV 데이터를 기반으로 모멘텀, 시장 관심도, 추세 등의 판단과 예측을 수행할 수 있습니다.

## Data for a single ticker

Here we provide two ways to fetch data with single ticker, let's take Apple Inc. (AAPL) as an example.

### Using yfinance

In [2]:
TRAIN_START_DATE = '2020-01-01'
TRADE_END_DATE = '2020-01-31'
aapl_df_yf = yf.download(tickers = "aapl", start=TRAIN_START_DATE, end=TRADE_END_DATE)

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


In [3]:
aapl_df_yf.head()

Price,Close,High,Low,Open,Volume
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL
Date,,,,,
2020-01-02,72.620827,72.681274,71.373203,71.627077,135480400
2020-01-03,71.914818,72.676447,71.689957,71.847118,146322800
2020-01-06,72.487862,72.526549,70.783263,71.034724,118387200
2020-01-07,72.146927,72.753808,71.926900,72.497514,108872000
2020-01-08,73.307541,73.609775,71.849563,71.849563,132079200


### Using FinRL

In FinRL's YahooDownloader, we modified the data frame to the form that convenient for further data processing process. We use adjusted close price instead of close price, and add a column representing the day of a week (0-4 corresponding to Monday-Friday).

In [4]:
aapl_df_finrl = YahooDownloader(start_date = TRAIN_START_DATE,
                                end_date = TRAIN_END_DATE,
                                ticker_list = ['aapl']).fetch_data()

YF deprecation warning: set proxy via new config function: yf.set_config(proxy=proxy)


[*********************100%***********************]  1 of 1 completed

Shape of DataFrame:  (146, 8)


In [5]:
aapl_df_finrl.head()

Price,date,close,high,low,open,volume,tic,day
0,2020-01-02,72.620834,72.681281,71.373211,71.627084,135480400,aapl,3
1,2020-01-03,71.914810,72.676439,71.689950,71.847110,146322800,aapl,4
2,2020-01-06,72.487854,72.526541,70.783256,71.034717,118387200,aapl,0
3,2020-01-07,72.146942,72.753823,71.926915,72.497529,108872000,aapl,1
4,2020-01-08,73.307510,73.609745,71.849533,71.849533,132079200,aapl,2


## Data for the chosen tickers

In [6]:
config_tickers.DOW_30_TICKER

['AXP',
 'AMGN',
 'AAPL',
 'BA',
 'CAT',
 'CSCO',
 'CVX',
 'GS',
 'HD',
 'HON',
 'IBM',
 'INTC',
 'JNJ',
 'KO',
 'JPM',
 'MCD',
 'MMM',
 'MRK',
 'MSFT',
 'NKE',
 'PG',
 'TRV',
 'UNH',
 'CRM',
 'VZ',
 'V',
 'WBA',
 'WMT',
 'DIS',
 'DOW']

In [7]:
TRAIN_START_DATE = '2009-01-01'
TRAIN_END_DATE = '2020-01-01'

TRADE_START_DATE = '2020-01-01'
TRADE_END_DATE = '2025-05-16'

In [8]:
df_raw = YahooDownloader(start_date = TRAIN_START_DATE,
                     end_date = TRADE_END_DATE,
                     ticker_list = config_tickers.DOW_30_TICKER).fetch_data()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

Shape of DataFrame:  (120971, 8)


In [9]:
df_raw.head()

Price,date,close,high,low,open,volume,tic,day
0,2009-01-02,2.727417,2.736133,2.559415,2.581053,746015200,AAPL,4
1,2009-01-02,41.151958,41.214743,40.286922,40.872914,6547900,AMGN,4
2,2009-01-02,14.929293,15.076037,14.211018,14.342316,10955700,AXP,4
3,2009-01-02,33.941090,34.173615,32.088393,32.103395,7010200,BA,4
4,2009-01-02,30.344683,30.389963,28.921568,29.050942,7117200,CAT,4


# Part 3: Preprocess Data
We need to check for missing data and do feature engineering to convert the data point into a state.
* **Adding technical indicators**. In practical trading, various information needs to be taken into account, such as historical prices, current holding shares, technical indicators, etc. Here, we demonstrate two trend-following technical indicators: MACD and RSI.
* **Adding turbulence index**. Risk-aversion reflects whether an investor prefers to protect the capital. It also influences one's trading strategy when facing different market volatility level. To control the risk in a worst-case scenario, such as financial crisis of 2007–2008, FinRL employs the turbulence index that measures extreme fluctuation of asset price.

# Part 3: 데이터 전처리
우리는 결측값이 있는지 확인하고, 데이터를 상태(state)로 변환하기 위한 피처 엔지니어링을 수행해야 합니다.

기술적 지표 추가: 실제 트레이딩에서는 과거 가격, 현재 보유 주식 수, 기술적 지표 등 다양한 정보를 고려해야 합니다.
여기서는 추세 추종 계열의 기술적 지표 두 가지, MACD와 RSI를 예제로 사용합니다.

변동성 지수(Turbulence Index) 추가:
위험 회피 성향은 투자자가 자본을 보호하려는 경향을 반영하며, 이는 시장 변동성 수준에 따라 거래 전략에 영향을 줍니다.
2007–2008년 금융 위기와 같은 최악의 시나리오에서 리스크를 통제하기 위해, FinRL은 자산 가격의 극단적 변동성을 측정하는 **변동성 지수(Turbulence Index)**를 활용합니다.

Hear let's take **MACD** as an example. Moving average convergence/divergence (MACD) is one of the most commonly used indicator showing bull and bear market. Its calculation is based on EMA (Exponential Moving Average indicator, measuring trend direction over a period of time.)

In [10]:
fe = FeatureEngineer(use_technical_indicator=True,
                     tech_indicator_list = INDICATORS,
                     use_vix=True,
                     use_turbulence=True,
                     user_defined_feature = False)

processed = fe.preprocess_data(df_raw)

Successfully added technical indicators


[*********************100%***********************]  1 of 1 completed


Shape of DataFrame:  (4117, 8)
Successfully added vix
Successfully added turbulence index


In [11]:
list_ticker = processed["tic"].unique().tolist()
list_date = list(pd.date_range(processed['date'].min(),processed['date'].max()).astype(str))
combination = list(itertools.product(list_date,list_ticker))

processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left")
processed_full = processed_full[processed_full['date'].isin(processed['date'])]
processed_full = processed_full.sort_values(['date','tic'])

processed_full = processed_full.fillna(0)

In [12]:
import matplotlib.pyplot as plt

In [13]:
processed_full

,date,tic,close,high,low,open,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2009-01-02,AAPL,2.727417,2.736133,2.559415,2.581053,746015200.0,4.0,0.000000,2.947758,2.622183,100.000000,66.666667,100.000000,2.727417,2.727417,39.189999,0.000000
1,2009-01-02,AMGN,41.151958,41.214743,40.286922,40.872914,6547900.0,4.0,0.000000,2.947758,2.622183,100.000000,66.666667,100.000000,41.151958,41.151958,39.189999,0.000000
2,2009-01-02,AXP,14.929293,15.076037,14.211018,14.342316,10955700.0,4.0,0.000000,2.947758,2.622183,100.000000,66.666667,100.000000,14.929293,14.929293,39.189999,0.000000
3,2009-01-02,BA,33.941090,34.173615,32.088393,32.103395,7010200.0,4.0,0.000000,2.947758,2.622183,100.000000,66.666667,100.000000,33.941090,33.941090,39.189999,0.000000
4,2009-01-02,CAT,30.344683,30.389963,28.921568,29.050942,7117200.0,4.0,0.000000,2.947758,2.622183,100.000000,66.666667,100.000000,30.344683,30.344683,39.189999,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173328,2025-05-14,UNH,308.010010,322.940002,307.109985,321.450012,40029600.0,2.0,-43.160651,517.574599,298.274400,25.754153,-130.122966,65.678971,458.932668,475.628676,18.620001,21.419092
173329,2025-05-14,V,356.459991,358.410004,355.209991,356.670013,6207600.0,2.0,5.492626,362.673423,321.380864,58.046238,130.023864,29.006548,337.199104,340.894076,18.620001,21.419092
173330,2025-05-14,VZ,42.349998,42.939999,42.250000,42.779999,20447500.0,2.0,-0.038602,44.491301,41.951700,48.630680,-65.617684,27.217860,43.225456,43.181627,18.620001,21.419092
173331,2025-05-14,WBA,11.130000,11.370000,11.130000,11.360000,13965000.0,2.0,0.083917,11.341615,10.726385,52.870720,118.592370,9.869636,10.963667,10.990000,18.620001,21.419092


In [14]:
processed_full[processed_full.tic=='WBA']['rsi_30'].head(10)


27     100.000000
114    100.000000
143     90.558993
172     92.388800
201     82.560288
230     58.986851
317     53.284206
346     56.258890
375     57.413840
404     67.155392
Name: rsi_30, dtype: float64

# Part 4: Save the Data

### Split the data for training and trading

In [15]:
train = data_split(processed_full, TRAIN_START_DATE,TRAIN_END_DATE)
trade = data_split(processed_full, TRADE_START_DATE,TRADE_END_DATE)
print(len(train))
print(len(trade))

80272
39121


### Save data to csv file

For Colab users, you can open the virtual directory in colab and manually download the files.

For users running on your local environment, the csv files should be at the same directory of this notebook.

In [16]:
train.to_csv('train_data.csv')
trade.to_csv('trade_data.csv')